In [ ]:
%matplotlib inline

import sys, os 
sys.path.append(os.path.join('..','..')) 
os.chdir('/Users/timcornwell/Code/crocodile/')
print(os.getcwd())

import pylab
pylab.rcParams['figure.figsize'] = (12.0, 12.0)
pylab.rcParams['image.cmap'] = 'rainbow'

from astropy import constants as const
import scipy.special

from matplotlib import pylab
from matplotlib import pyplot as plt

from crocodile.synthesis import *
from crocodile.simulate import *
from util.visualize import *
from crocodile.msclean import *

from functions.imaging import *
from functions.image import image_show

In [ ]:
kwargs = {}

vlaa = configuration_filter(named_configuration('VLAA'), **kwargs)
vlaa.data['xyz']=vlaa.data['xyz']/30.0

In [ ]:
times = numpy.arange(-numpy.pi/2.0, +numpy.pi/2.0,0.05)
frequency = numpy.arange(1.0e8, 1.50e8, 1e6)
frequency = 1.0e8*numpy.ones([1])
reffrequency = numpy.average(frequency)
direction = SkyCoord(0.0*u.rad, u.rad*numpy.pi/4, frame='icrs')
vt = simulate(vlaa, times, frequency, weight=1.0, direction=direction)

In [ ]:
plt.clf()
for f in frequency:
    x=f/const.c
    plt.plot(x*vt.data['uvw'][:,0], x*vt.data['uvw'][:,1], '.', color='b')
    plt.plot(-x*vt.data['uvw'][:,0], -x*vt.data['uvw'][:,1], '.', color='r')

In [ ]:
m31image = image_from_fits("./data/models/m31.MOD")
fig = plt.figure()
fig.add_subplot(111, projection=m31image.wcs)
plt.imshow(m31image.data, origin='lower', cmap='rainbow')
plt.xlabel('RA---SIN')
plt.ylabel('DEC--SIN')
plt.show()

In [ ]:
m31image4D=image_replicate(m31image, shape=[1, 1, 1, len(frequency)])
m31sm = skymodel_from_image(m31image4D)

In [ ]:
vtpred = simulate(vlaa, times, frequency, weight=1.0, direction=direction)

In [ ]:
kwargs={'wstep':30.0}
vtpred = predict(vtpred, m31sm, **kwargs)

In [ ]:
uvdist=numpy.sqrt(vtpred.data['uvw'][:,0]**2+vtpred.data['uvw'][:,1]**2)
plt.clf()
plt.plot(uvdist, numpy.abs(vtpred.data['vis'][:,0,0]), '.')
plt.xlabel('uvdist')
plt.ylabel('Amp Visibility')
plt.show()

In [ ]:
kwargs={}
kwargs['npixel']=512
kwargs['cellsize']=0.0004
kwargs['wstep']=30.0
dirty, psf, sumwt = invert(vtpred, **kwargs)

In [ ]:
image_show(dirty)

In [ ]:
print("Max, min in dirty image = %.6f, %.6f, sum of weights = %f" % (dirty.data.max(), dirty.data.min(), sumwt))
image_to_fits(dirty, 'dirty.fits')
image_to_fits(psf, 'psf.fits')
m31smnew, res = fitcomponent(dirty, m31sm, **kwargs)